In [3]:
# load the data from API 
import pandas as pd
from sodapy import Socrata
from tqdm import tqdm
import time
import requests

In [9]:
# Loading 311 data for New York City 
client = Socrata("data.cityofnewyork.us", None)
DATASET_ID = "erm2-nwe9"
CHUNK_SIZE = 10000         
MAX_RECORDS = 100000       
OUTPUT_FILE = "nyc_311_sample.csv"

all_results = []

for offset in tqdm(range(0, MAX_RECORDS, CHUNK_SIZE)):
    try:
        results = client.get(
            DATASET_ID,
            limit=CHUNK_SIZE,
            offset=offset,
            order="created_date DESC"  # newest first
        )
        if not results:
            break
        df_chunk = pd.DataFrame.from_records(results)
        all_results.append(df_chunk)

        time.sleep(0.2)

    except Exception as e:
        print(f"Error at offset {offset}: {e}")
        break

if all_results:
    full_df = pd.concat(all_results, ignore_index=True)
    full_df.to_csv(OUTPUT_FILE, index=False)
    print(f"Saved {len(full_df)} rows to {OUTPUT_FILE}")
else:
    print("No data retrieved.")

100%|██████████| 10/10 [00:12<00:00,  1.28s/it]


✅ Saved 100000 rows to nyc_311_sample.csv


In [30]:
# Viewing 311 data
nyc_311 = pd.read_csv("nyc_311_sample.csv")
nyc_311.info

<bound method DataFrame.info of        unique_key             created_date agency  \
0        64720753  2025-04-23T01:51:16.000   NYPD   
1        64725906  2025-04-23T01:51:14.000   NYPD   
2        64722271  2025-04-23T01:51:09.000   NYPD   
3        64727864  2025-04-23T01:50:41.000   NYPD   
4        64725747  2025-04-23T01:50:06.000   NYPD   
...           ...                      ...    ...   
99995    64620790  2025-04-11T17:38:10.000   DSNY   
99996    64623847  2025-04-11T17:37:51.000    HPD   
99997    64621478  2025-04-11T17:37:47.000   NYPD   
99998    64621068  2025-04-11T17:37:45.000   NYPD   
99999    64619917  2025-04-11T17:37:44.000    DOB   

                                             agency_name  \
0                        New York City Police Department   
1                        New York City Police Department   
2                        New York City Police Department   
3                        New York City Police Department   
4                        New Yo

In [31]:
nyc_311.head()

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,...,resolution_description,closed_date,taxi_pick_up_location,facility_type,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,taxi_company_borough,due_date
0,64720753,2025-04-23T01:51:16.000,NYPD,New York City Police Department,Noise - Vehicle,Car/Truck Music,Street/Sidewalk,11208.0,2858 LINDEN BOULEVARD,LINDEN BOULEVARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,64725906,2025-04-23T01:51:14.000,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,11206.0,406 LORIMER STREET,LORIMER STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,64722271,2025-04-23T01:51:09.000,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,Street/Sidewalk,11205.0,125 NORTH PORTLAND AVENUE,NORTH PORTLAND AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,64727864,2025-04-23T01:50:41.000,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,10461.0,2873 EAST 195 STREET,EAST 195 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,64725747,2025-04-23T01:50:06.000,NYPD,New York City Police Department,Drinking,After Hours - Licensed Est,Club/Bar/Restaurant,10458.0,2447 HOFFMAN STREET,HOFFMAN STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# Convert 'created_date' to datetime
nyc_311['created_date'] = pd.to_datetime(
    nyc_311['created_date'], errors='coerce'
)

# Filter for 2025 only
nyc_311_2025 = nyc_311[
    nyc_311['created_date'].dt.year == 2025
]

# Select only necessary columns
cols_to_keep = [
    'created_date', 'borough', 'incident_zip', 'complaint_type',
    'descriptor', 'location_type', 'agency', 'latitude', 'longitude'
]

nyc_311_filtered = nyc_311_2025[cols_to_keep].copy()

# Drop rows without incident_zip and standardize ZIP codes
nyc_311_filtered = nyc_311_filtered.dropna(subset=['incident_zip'])
nyc_311_filtered['incident_zip'] = (
    nyc_311_filtered['incident_zip'].astype(int).astype(str).str.zfill(5)
)

print(nyc_311_filtered['created_date'].dt.year.value_counts())
print(nyc_311_filtered['incident_zip'].unique()[:10])
print(nyc_311_filtered.info())

created_date
2025    99029
Name: count, dtype: int64
['11208' '11206' '11205' '10461' '10458' '11412' '11435' '11368' '11374'
 '11226']
<class 'pandas.core.frame.DataFrame'>
Index: 99029 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   created_date    99029 non-null  datetime64[ns]
 1   borough         99029 non-null  object        
 2   incident_zip    99029 non-null  object        
 3   complaint_type  99029 non-null  object        
 4   descriptor      96704 non-null  object        
 5   location_type   88480 non-null  object        
 6   agency          99029 non-null  object        
 7   latitude        97812 non-null  float64       
 8   longitude       97812 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 7.6+ MB
None


In [37]:
nyc_311_filtered.head()

,created_date,borough,incident_zip,complaint_type,descriptor,location_type,agency,latitude,longitude
0,2025-04-23 01:51:16,BROOKLYN,11208,Noise - Vehicle,Car/Truck Music,Street/Sidewalk,NYPD,40.670044,-73.858687
1,2025-04-23 01:51:14,BROOKLYN,11206,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,NYPD,40.709122,-73.948547
2,2025-04-23 01:51:09,BROOKLYN,11205,Illegal Parking,Posted Parking Sign Violation,Street/Sidewalk,NYPD,40.694353,-73.976030
3,2025-04-23 01:50:41,BRONX,10461,Illegal Parking,Blocked Hydrant,Street/Sidewalk,NYPD,40.853187,-73.832102
4,2025-04-23 01:50:06,BRONX,10458,Drinking,After Hours - Licensed Est,Club/Bar/Restaurant,NYPD,40.857248,-73.887390


In [38]:
print("First date:", nyc_311_filtered['created_date'].min())
print("Last date:", nyc_311_filtered['created_date'].max())

First date: 2025-04-11 17:37:44
Last date: 2025-04-23 01:51:16


In [70]:
# Get a lookup table for zip -> borough
zip_borough = nyc_311_filtered[['incident_zip', 'borough']].drop_duplicates(subset='incident_zip')

# pivot_table
zip_total_complaints = nyc_311_filtered.groupby('incident_zip').size().reset_index(name='total_complaints')

zip_complaint_counts = nyc_311_filtered.pivot_table(
    index='incident_zip',
    columns='complaint_type',
    aggfunc='size',
    fill_value=0
).reset_index()

zip_summary = zip_total_complaints.merge(zip_complaint_counts, on='incident_zip')

top_complaint_per_zip = nyc_311_filtered.groupby(['incident_zip', 'complaint_type']).size()
top_complaint_per_zip = top_complaint_per_zip.reset_index(name='count')
top_complaint_per_zip = top_complaint_per_zip.sort_values(['incident_zip', 'count'], ascending=[True, False])
top_complaint_per_zip = top_complaint_per_zip.drop_duplicates('incident_zip')
top_complaint_per_zip = top_complaint_per_zip.rename(columns={'complaint_type': 'top_complaint', 'count': 'top_complaint_count'})

zip_summary = zip_summary.merge(top_complaint_per_zip[['incident_zip', 'top_complaint', 'top_complaint_count']], on='incident_zip')

# merge back borough info
zip_summary = zip_summary.merge(zip_borough, on='incident_zip', how='left')


In [71]:
zip_summary.head()
print(zip_summary.head())
print(zip_summary.columns)
[col for col in zip_summary.columns if 'borough' in col.lower()]

  incident_zip  total_complaints  AHV Inspection Unit  APPLIANCE  \
0        10000                24                    0          0   
1        10001               632                    0          0   
2        10002               856                    0          4   
3        10003               686                    0          2   
4        10004               265                    0          0   

   Abandoned Bike  Abandoned Vehicle  Air Quality  \
0               0                  0            0   
1               2                  1            1   
2               2                  5            1   
3               1                  0            4   
4               0                  1            0   

   Animal Facility - No Permit  Animal in a Park  Animal-Abuse  ...  \
0                            0                 0             0  ...   
1                            0                 0             4  ...   
2                            0                 8           

['borough']

In [3]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv('CENSUS_API_KEY')

# Define variables to request
variables = [
    'B01003_001E',  # Total population
    'B19013_001E',  # Median household income
    'B17001_001E',  # Poverty universe (denominator)
    'B17001_002E',  # Below poverty line (numerator)
    'B02001_002E',  # White alone
    'B02001_003E',  # Black or African American alone
    'B03003_003E'   # Hispanic or Latino
]

# Build URL
var_str = ','.join(variables)
url = f'https://api.census.gov/data/2020/acs/acs5?get=NAME,{var_str}&for=zip%20code%20tabulation%20area:*&key={api_key}'

# Fetch and load data
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data[1:], columns=data[0])

# Rename for clarity
census = df.rename(columns={
    'zip code tabulation area': 'incident_zip',
    'B01003_001E': 'total_population',
    'B19013_001E': 'median_income',
    'B17001_001E': 'poverty_universe',
    'B17001_002E': 'below_poverty',
    'B02001_002E': 'white_alone',
    'B02001_003E': 'black_alone',
    'B03003_003E': 'hispanic'
})

# Format ZIP codes to 5-digit strings
census['incident_zip'] = census['incident_zip'].str.zfill(5)

# Convert to numeric and calculate poverty rate
for col in ['total_population', 'median_income', 'poverty_universe', 'below_poverty',
            'white_alone', 'black_alone', 'hispanic']:
    census[col] = pd.to_numeric(census[col], errors='coerce')

census['poverty_rate'] = census['below_poverty'] / census['poverty_universe']

print(census.head())

          NAME  total_population  median_income  poverty_universe  \
0  ZCTA5 29590              3543          30985              2341   
1  ZCTA5 93306             74296          54450             73646   
2  ZCTA5 93660              4082          39625              4082   
3  ZCTA5 93110             15777          93264             14646   
4  ZCTA5 93212             22596          42983             13554   

   below_poverty  white_alone  black_alone  hispanic incident_zip  \
0            869          718         2617       346        29590   
1          15853        51869         1727     48116        93306   
2           1057         2231           40      3925        93660   
3           1332        12042          311      4251        93110   
4           3911        13033         2708     15641        93212   

   poverty_rate  
0      0.371209  
1      0.215259  
2      0.258942  
3      0.090946  
4      0.288550  


In [54]:
census.info

<bound method DataFrame.info of               NAME  total_population  median_income  poverty_universe  \
0      ZCTA5 29590              3543          30985              2341   
1      ZCTA5 93306             74296          54450             73646   
2      ZCTA5 93660              4082          39625              4082   
3      ZCTA5 93110             15777          93264             14646   
4      ZCTA5 93212             22596          42983             13554   
...            ...               ...            ...               ...   
33115  ZCTA5 16623               552          51667               552   
33116  ZCTA5 16627              2118          45000              2097   
33117  ZCTA5 16634               315          51500               309   
33118  ZCTA5 16640               707          55982               707   
33119  ZCTA5 17062              4289          73036              4286   

       below_poverty  white_alone  black_alone  hispanic incident_zip  \
0                8

In [72]:
merged_df = zip_summary.merge(census, on='incident_zip', how='left')
merged_df.info

<bound method DataFrame.info of     incident_zip  total_complaints  AHV Inspection Unit  APPLIANCE  \
0          10000                24                    0          0   
1          10001               632                    0          0   
2          10002               856                    0          4   
3          10003               686                    0          2   
4          10004               265                    0          0   
..           ...               ...                  ...        ...   
211        11697                 1                    0          0   
212        11801                 1                    0          0   
213        12345                 1                    0          0   
214        19013                 1                    0          0   
215        55126                 1                    0          0   

     Abandoned Bike  Abandoned Vehicle  Air Quality  \
0                 0                  0            0   
1                

In [73]:
merged_df.to_csv('./zip_summary.csv', index=False)